In [1]:
import os
# Change native directory to root
os.chdir(os.path.dirname(os.getcwd()))

In [7]:
import glob
import pandas as pd
import numpy as np
import random
from tsfeatures import tsfeatures

random.seed(123)
model_dir = 'models/'

In [10]:
meta_data = pd.read_csv('data/EANLIJST_METADATA.csv', sep=';')
build_list = glob.glob('./data/buildings/*.csv')
test_list = random.sample(build_list, int(len(build_list) * 0.1))
train_list = [x for x in build_list if x not in test_list]
meta_data

,EAN_ID,Patrimonium Type,Patrimonium Functietype,Meetmethode,Richting
0,681,NaN,NaN,Continu,Afname
1,125,NaN,NaN,Continu,Afname
2,213,NaN,NaN,Continu,Afname
3,603,NaN,NaN,Continu,Afname
4,1699,NaN,NaN,Continu,Afname
...,...,...,...,...,...
1736,963,Gebouw,Sportcomplex,Continu,Afname
1737,1364,NaN,NaN,Continu,Afname
1738,473,Gebouw,Werkplaats,Continu,Afname
1739,965,Gebouw,Administratief centrum,Continu,Afname


In [6]:
volume_data = pd.read_csv('./data/EAN_VOLUME.csv')
volume_data

,unique_id,ds,y
0,205,2019-01-01 00:15:00,3.34
1,205,2019-01-01 00:30:00,3.34
2,205,2019-01-01 00:45:00,3.34
3,205,2019-01-01 01:00:00,3.34
4,205,2019-01-01 01:15:00,3.34
...,...,...,...
183181051,959,2021-12-31 23:00:00,0.48
183181052,959,2021-12-31 23:15:00,0.54
183181053,959,2021-12-31 23:30:00,0.66
183181054,959,2021-12-31 23:45:00,0.66


In [7]:
# Save time series for each EAN_ID into a buildings folder as a csv file
for ean in volume_data.unique_id.unique():
    ean_data = volume_data[volume_data.unique_id == ean]
    # Read datetime string 01JAN19:00:15:00 to datetimeindex
    ean_data.index = pd.to_datetime(ean_data['ds'], format='%d%b%y:%H:%M:%S')
    # rename volume to kwh
    ean_data = ean_data.rename(columns={'y': 'Power'})
    # keep only kwh column and index
    ean_data = ean_data[['Power']]
    ean_data['function'] = meta_data.loc[meta_data.EAN_ID == ean, 'function'].values[0]
    ean_data.to_csv(f'data/buildings/{ean}.csv')

AttributeError: 'DataFrame' object has no attribute 'EAN_ID'

In [8]:
# Show unique value counts of each feature in each column of meta_data
for col in meta_data.columns:
    print(col, meta_data[col].value_counts())

EAN_ID 681     1
1236    1
614     1
1246    1
297     1
       ..
302     1
1263    1
301     1
1259    1
241     1
Name: EAN_ID, Length: 1741, dtype: int64
Patrimonium Type Gebouw                  912
Andere                   30
Terrein                   8
Openbare verlichting      5
Name: Patrimonium Type, dtype: int64
Patrimonium Functietype Sporthal                                    116
Andere gebouwen                              99
Administratief centrum                       91
Cultureel centrum                            84
Ontmoetingscentrum                           73
Lagere school                                61
RVT/WZC/revalidatiecentrum                   47
Stadhuis/Gemeentehuis                        42
Werkplaats                                   33
Sportcomplex                                 32
Bibliotheek                                  29
Museum                                       27
Academie                                     25
Kast                        

In [4]:
filename = build_list[0]
ts = pd.read_csv(filename, usecols=['Power', 'ds'], index_col='ds')
ts.index = pd.to_datetime(ts.index)
# divide by yearly consumption
#ts = ts / ts.resample('A').sum().values[1]
# groupby hour of the day
hourly = ts.groupby(ts.index.hour).mean() / ts.groupby(ts.index.hour).mean().sum().to_dict()
# group by day of the week
weekly = ts.groupby(ts.index.weekday).mean() / ts.groupby(ts.index.weekday).mean().sum().to_dict()
# add a letter in front of index
hourly.index = ['h' + str(i) for i in hourly.index]
weekly.index = ['w' + str(i) for i in weekly.index]
# merge hourly and weekly into a dataframe with index as column names
hourly_weekly = pd.DataFrame(pd.concat([hourly, weekly], axis=0)).T
hourly_weekly.index = [filename.split('.')[1].split('/')[-1]]
# add yearly consumption to dataframe
hourly_weekly['yearly'] = ts.resample('A').sum().values[1]
#hourly_weekly
ts.resample('A').sum()

,Power
ds,
2019-12-31,15870946.0
2020-12-31,13085002.0
2021-12-31,12945184.0
2022-12-31,217.0


In [12]:
output_path='./data/social_features.csv'
for filename in train_list:
    ts = pd.read_csv(filename, usecols=['Power', 'ds'], index_col='ds')
    ts.index = pd.to_datetime(ts.index)
    # groupby hour of the day
    hourly = ts.groupby(ts.index.hour).mean() / ts.groupby(ts.index.hour).mean().sum().to_dict()
    # group by day of the week
    weekly = ts.groupby(ts.index.weekday).mean() / ts.groupby(ts.index.weekday).mean().sum().to_dict()
    # add a letter in front of index
    hourly.index = ['h' + str(i) for i in hourly.index]
    weekly.index = ['w' + str(i) for i in weekly.index]
    # merge hourly and weekly into a dataframe with index as column names
    hourly_weekly = pd.DataFrame(pd.concat([hourly, weekly], axis=0)).T
    hourly_weekly.index = [filename.split('.')[1].split('/')[-1]]
    # add yearly consumption to dataframe
    hourly_weekly['yearly'] = ts.resample('A').sum().values[0]
    hourly_weekly.to_csv(output_path, mode='a', header=not os.path.exists(output_path))


In [ ]:
from catch22 import catch22_all
output_path='./data/features.csv'
for filename in build_list:
    ts = pd.read_csv(filename, usecols=['Power', 'ds'], index_col='ds')
    c22 = catch22_all(ts['Power'])
    c22_df = pd.DataFrame(data=[c22['values']], columns=c22['names'], index=[filename.split('.')[1].split('/')[-1]])
#     c22['mean'] = ts.mean()
#     c22['std'] = ts.std()
#     c22['ID'] = filename.split('.')[1].split('/')[-1]
    c22_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))

In [ ]:
# calculate mean and std for each building
ts_metrics = pd.DataFrame(columns=['mean', 'std', 'ID'])
for filename in build_list:
    ts = pd.read_csv(filename, usecols=['Power', 'ds'], index_col='ds')
    ts_metrics.loc[filename.split('.')[1].split('/')[-1]] = [ts.mean(), ts.std(), filename.split('.')[1].split('/')[-1]]
# Save to csv